# FastAPI upload server (payload_video.ipynb)

Notebook ini menyediakan server FastAPI yang menerima upload video (multipart) di `/upload` dan menerima JSON payload di `/upload`.

Langkah eksekusi:
1. Jalankan cell instalasi dependensi
2. Jalankan cell setup direktori
3. Jalankan cell definisi server
4. Jalankan cell start server (ngrok akan dicoba jika tersedia)

Hasil: file yang diupload akan disimpan di folder `uploads/` dan payload JSON yang dikirim ke `/upload` akan disimpan di `received_payloads/`. Video akan diproses dengan Whisper untuk speech-to-text.

In [1]:
# Install dependencies (jalankan sekali)
!pip install --quiet fastapi uvicorn nest-asyncio pyngrok python-multipart
!pip install --quiet faster-whisper
!pip install --quiet tqdm
!pip install --quiet imageio-ffmpeg
!pip install --quiet deepl

print('\n✅ All packages installed successfully')
print('ℹ️  Using faster-whisper (4-5x faster than openai-whisper)')
print('ℹ️  Using DeepL for Indonesian translation')


✅ All packages installed successfully
ℹ️  Using faster-whisper (4-5x faster than openai-whisper)
ℹ️  Using DeepL for Indonesian translation


In [2]:
# Siapkan direktori untuk upload dan transcription
import os
import sys
import shutil

# Setup directories
ROOT_DIR = os.getcwd()
UPLOAD_DIR = os.path.join(ROOT_DIR, 'uploads')
TRANSCRIPTION_DIR = os.path.join(ROOT_DIR, 'transcriptions')
RESULTS_DIR = os.path.join(ROOT_DIR, 'results')  # NEW: hasil assessment
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(TRANSCRIPTION_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print('📁 Directories:')
print(f'   Upload: {UPLOAD_DIR}')
print(f'   Transcription: {TRANSCRIPTION_DIR}')
print(f'   Results: {RESULTS_DIR}')

# Check for GPU
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

print(f'\n🎯 Device Configuration:')
print(f'   Device: {device.upper()}')
print(f'   Compute Type: {compute_type}')
if device == "cuda":
    print(f'   GPU: {torch.cuda.get_device_name(0)}')
else:
    print('   Note: Using CPU (GPU recommended for faster processing)')

# DeepL Configuration
DEEPL_API_KEY = "02a88edf-4fcb-4786-ba3d-a137fb143760:fx"

print('\n🌐 Translation Configuration:')
print(f'   DeepL API: {"Configured" if DEEPL_API_KEY != "YOUR_DEEPL_API_KEY_HERE" else "⚠️  NOT CONFIGURED - Set DEEPL_API_KEY"}')

📁 Directories:
   Upload: d:\Coding\Interview_Assesment_System-main\uploads
   Transcription: d:\Coding\Interview_Assesment_System-main\transcriptions
   Results: d:\Coding\Interview_Assesment_System-main\results

🎯 Device Configuration:
   Device: CPU
   Compute Type: int8
   Note: Using CPU (GPU recommended for faster processing)

🌐 Translation Configuration:
   DeepL API: Configured

🎯 Device Configuration:
   Device: CPU
   Compute Type: int8
   Note: Using CPU (GPU recommended for faster processing)

🌐 Translation Configuration:
   DeepL API: Configured


In [ ]:
from fastapi import FastAPI, UploadFile, File, Form, Request, HTTPException, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, HTMLResponse
from fastapi.staticfiles import StaticFiles
import uuid, shutil, json, os, sys
from datetime import datetime, timezone
import urllib.request
import tempfile
from tqdm import tqdm
import hashlib
import time
from urllib.parse import urlparse, parse_qs
import subprocess
from typing import List
import random
from faster_whisper import WhisperModel
import torch
import deepl
import requests

app = FastAPI(title='AI Interview Assessment System')

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
    expose_headers=['*'],
    max_age=3600,
)

# Mount static folders
app.mount('/uploads', StaticFiles(directory=UPLOAD_DIR), name='uploads')
app.mount('/transcriptions', StaticFiles(directory=TRANSCRIPTION_DIR), name='transcriptions')
app.mount('/results', StaticFiles(directory=RESULTS_DIR), name='results')

# Load faster-whisper model with BEST ACCURACY settings
print('\n📥 Loading Whisper model...')
print('ℹ️  Using faster-whisper "large-v3" model')
print('   This is the MOST ACCURATE model available')
print('   Speed: 4-5x faster than openai-whisper')
print('   Accuracy: ~98% for clear English speech')
print('   First run will download ~3GB model...\n')

# Detect device
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

print(f'🎯 Configuration:')
print(f'   Device: {device.upper()}')
print(f'   Compute Type: {compute_type}')

# Load model with best accuracy settings
whisper_model = WhisperModel(
    "large-v3",
    device=device,
    compute_type=compute_type,
    cpu_threads=4,
    num_workers=1
)

print('✅ Whisper model loaded successfully\n')

# Initialize DeepL translator
translator = None
if DEEPL_API_KEY and DEEPL_API_KEY != "YOUR_DEEPL_API_KEY_HERE":
    try:
        translator = deepl.Translator(DEEPL_API_KEY)
        print('✅ DeepL translator initialized successfully\n')
    except Exception as e:
        print(f'⚠️  DeepL initialization failed: {e}')
        print('   Translation to Indonesian will be skipped\n')
else:
    print('⚠️  DeepL API key not configured')
    print('   Translation to Indonesian will be skipped\n')

# Background processing
import threading
import threading as th
from concurrent.futures import ThreadPoolExecutor

executor = ThreadPoolExecutor(max_workers=2)
processing_status = {}
processing_lock = th.Lock()

# HELPER FUNCTIONS

def get_local_file_path(url):
    """Extract local file path from URL if it's a local upload"""
    try:
        parsed = urlparse(url)
        if '/uploads/' in parsed.path:
            filename = parsed.path.split('/uploads/')[-1]
            local_path = os.path.join(UPLOAD_DIR, filename)
            if os.path.exists(local_path):
                return local_path
    except Exception as e:
        print(f'Error parsing URL: {e}')
    return None

def download_video_from_url(url, position_id):
    """Download video from external URL (Google Drive, etc)"""
    try:
        print(f'   📥 Downloading video from URL...')
        
        # Handle Google Drive links
        if 'drive.google.com' in url:
            # Extract file ID from Google Drive URL
            if '/file/d/' in url:
                file_id = url.split('/file/d/')[1].split('/')[0]
            elif 'id=' in url:
                file_id = parse_qs(urlparse(url).query).get('id', [None])[0]
            else:
                raise Exception("Cannot extract file ID from Google Drive URL")
            
            # Use Google Drive direct download URL
            download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
            
            print(f'   🔗 Google Drive file ID: {file_id}')
        else:
            download_url = url
        
        # Download with progress
        response = requests.get(download_url, stream=True, timeout=300)
        response.raise_for_status()
        
        # Determine file extension
        content_type = response.headers.get('Content-Type', '')
        if 'video/mp4' in content_type:
            ext = '.mp4'
        elif 'video/webm' in content_type:
            ext = '.webm'
        elif 'video/quicktime' in content_type:
            ext = '.mov'
        else:
            ext = '.mp4'  # Default
        
        # Save to temp file
        temp_filename = f"download_pos{position_id}_{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}{ext}"
        temp_path = os.path.join(UPLOAD_DIR, temp_filename)
        
        total_size = int(response.headers.get('content-length', 0))
        
        with open(temp_path, 'wb') as f:
            if total_size:
                downloaded = 0
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        downloaded += len(chunk)
                        percent = (downloaded / total_size) * 100
                        if downloaded % (1024 * 1024) == 0:  # Every 1MB
                            print(f'      Downloaded: {downloaded / (1024*1024):.1f}MB / {total_size / (1024*1024):.1f}MB ({percent:.1f}%)')
            else:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
        
        file_size_mb = os.path.getsize(temp_path) / (1024 * 1024)
        print(f'   ✅ Download complete: {file_size_mb:.2f} MB')
        
        return temp_path
        
    except requests.exceptions.Timeout:
        raise Exception("Download timeout - file terlalu besar atau koneksi lambat")
    except requests.exceptions.RequestException as e:
        raise Exception(f"Download failed: {str(e)}")
    except Exception as e:
        raise Exception(f"Download error: {str(e)}")

def get_video_file(video_url, position_id, is_external=False):
    """
    Unified function to get video file path
    - If local upload: extract from URL
    - If external: download from internet
    """
    if is_external:
        # Download from external URL
        return download_video_from_url(video_url, position_id)
    else:
        # Get local file path
        local_file = get_local_file_path(video_url)
        if not local_file:
            raise Exception(f"Local file not found for URL: {video_url}")
        return local_file

def transcribe_video(video_path):
    """Transcribe video using faster-whisper with MAXIMUM ACCURACY settings"""
    try:
        if not os.path.exists(video_path):
            raise Exception(f"Video file not found: {video_path}")
        
        if not os.access(video_path, os.R_OK):
            raise Exception(f"Video file is not readable: {video_path}")
        
        file_size = os.path.getsize(video_path) / (1024 * 1024)
        print(f'📁 Video: {os.path.basename(video_path)} ({file_size:.2f} MB)')
        
        print('🔄 Starting transcription...')
        start_time = time.time()
        
        # Dynamic parameters based on file size
        if file_size > 30:
            print('   ⚡ Large file - using balanced mode')
            beam_size = 3
            best_of = 3
        else:
            beam_size = 5
            best_of = 5
        
        # Transcribe
        segments, info = whisper_model.transcribe(
            video_path,
            language="en",
            task="transcribe",
            beam_size=beam_size,
            best_of=best_of,
            patience=2.0,
            length_penalty=1.0,
            repetition_penalty=1.0,
            temperature=0.0,
            compression_ratio_threshold=2.4,
            log_prob_threshold=-1.0,
            no_speech_threshold=0.6,
            condition_on_previous_text=True,
            initial_prompt="This is a professional interview conversation in clear English. The speaker is answering interview questions.",
            vad_filter=True,
            vad_parameters=dict(
                threshold=0.5,
                min_speech_duration_ms=250,
                max_speech_duration_s=float('inf'),
                min_silence_duration_ms=2000,
                speech_pad_ms=400
            ),
            word_timestamps=False,
            hallucination_silence_threshold=None
        )
        
        # Collect segments with progress bar
        print('   📝 Collecting segments...')
        transcription_text = ""
        segments_list = list(segments)  # Convert generator to list first
        
        # Progress bar for segment collection
        for segment in tqdm(segments_list, desc="   Segments", unit="seg", ncols=80, leave=False):
            transcription_text += segment.text + " "
        
        transcription_text = transcription_text.strip()
        
        if not transcription_text:
            print('   ⚠️  No speech detected')
            return "[No speech detected in video]"
        
        total_time = time.time() - start_time
        words = transcription_text.split()
        
        print(f'   ✅ Completed in {total_time:.1f}s | {len(segments_list)} segments | {len(words)} words')
        
        # Cleanup
        import gc
        gc.collect()
        
        return transcription_text
            
    except Exception as e:
        import traceback
        print(f'   ❌ Error: {str(e)}')
        import gc
        gc.collect()
        raise Exception(f"Transcription failed: {str(e)}")

def translate_to_indonesian(text):
    """Translate English text to Indonesian using DeepL"""
    if not translator:
        print('   ⚠️  Translation skipped (no API key)')
        return "[Translation not available]"
    
    try:
        max_chunk_size = 5000
        
        if len(text) <= max_chunk_size:
            result = translator.translate_text(text, source_lang="EN", target_lang="ID")
            translated_text = result.text
        else:
            sentences = text.split('. ')
            translated_sentences = []
            current_chunk = ""
            
            # Progress bar for translation chunks
            for sentence in tqdm(sentences, desc="   Translation", unit="sent", ncols=80, leave=False):
                if len(current_chunk) + len(sentence) < max_chunk_size:
                    current_chunk += sentence + ". "
                else:
                    if current_chunk:
                        result = translator.translate_text(current_chunk.strip(), source_lang="EN", target_lang="ID")
                        translated_sentences.append(result.text)
                    current_chunk = sentence + ". "
            
            if current_chunk:
                result = translator.translate_text(current_chunk.strip(), source_lang="EN", target_lang="ID")
                translated_sentences.append(result.text)
            
            translated_text = " ".join(translated_sentences)
        
        print(f'   ✅ Translation: {len(text)} → {len(translated_text)} chars')
        return translated_text
        
    except Exception as e:
        print(f'   ❌ Translation failed: {str(e)}')
        return f"[Translation failed: {str(e)}]"

def generate_dummy_assessment(transcription_text, position_id, transcription_id=None):
    """Generate dummy assessment data untuk testing"""
    words = transcription_text.split()
    word_count = len(words)
    char_count = len(transcription_text)
    
    confidence_score = random.randint(85, 98)
    kualitas_jawaban = random.randint(80, 100)
    relevansi = random.randint(75, 95)
    koherensi = random.randint(70, 90)
    tempo_bicara = random.randint(80, 100)
    
    total = round((confidence_score + kualitas_jawaban + relevansi + koherensi + tempo_bicara) / 5)
    
    if total >= 90:
        penilaian_akhir = 5
    elif total >= 80:
        penilaian_akhir = 4
    elif total >= 70:
        penilaian_akhir = 3
    elif total >= 60:
        penilaian_akhir = 2
    else:
        penilaian_akhir = 1
    
    has_cheating = random.choice([True, False, False, False])
    
    if has_cheating:
        cheating_detection = "Ya"
        alasan_cheating = random.choice([
            "Terdeteksi adanya manipulasi suara",
            "Terdeteksi multiple speakers",
            "Pola jawaban tidak konsisten",
            "Kecepatan bicara tidak natural"
        ])
    else:
        cheating_detection = "Tidak"
        alasan_cheating = "Tidak ada indikasi kecurangan"
    
    analisis_options = [
        "Lancar dan tidak mencurigakan",
        "Sedikit gugup namun natural",
        "Sangat percaya diri",
        "Tempo bicara konsisten",
        "Artikulasi jelas"
    ]
    analisis_non_verbal = random.choice(analisis_options)
    
    if penilaian_akhir >= 4 and not has_cheating:
        keputusan_akhir = "Lulus"
    elif penilaian_akhir >= 3 and not has_cheating:
        keputusan_akhir = "Pertimbangan"
    else:
        keputusan_akhir = "Tidak Lulus"
    
    return {
        "penilaian": {
            "confidence_score": confidence_score,
            "kualitas_jawaban": kualitas_jawaban,
            "relevansi": relevansi,
            "koherensi": koherensi,
            "tempo_bicara": tempo_bicara,
            "total": total
        },
        "penilaian_akhir": penilaian_akhir,
        "cheating_detection": cheating_detection,
        "alasan_cheating": alasan_cheating,
        "analisis_non_verbal": analisis_non_verbal,
        "keputusan_akhir": keputusan_akhir,
        "transkripsi_en": transcription_text,
        "transkripsi_id": transcription_id,
        "metadata": {
            "word_count": word_count,
            "char_count": char_count,
            "processed_at": datetime.now(timezone.utc).isoformat(),
            "translation_available": transcription_id is not None  # NEW
        }
    }

# UNIFIED PROCESSING FUNCTION
def process_videos_unified(session_id: str, candidate_name: str, uploaded_videos: list, base_url: str):
    """
    UNIFIED background processing for videos
    Handles both local uploads and external URLs automatically
    """
    try:
        # Detect source type
        is_external = any(v.get('isExternal', False) for v in uploaded_videos)
        source_type = "EXTERNAL URLs" if is_external else "LOCAL UPLOADS"
        
        print(f'\n{"="*70}')
        print(f'🎙️  SESSION: {session_id} ({source_type})')
        print(f'👤 CANDIDATE: {candidate_name}')
        print(f'📹 VIDEOS: {len(uploaded_videos)}')
        print(f'{"="*70}\n')
        
        transcriptions = []
        assessment_results = []
        
        with processing_lock:
            processing_status[session_id] = {'status': 'processing', 'progress': '0/0'}
        
        # Process each video with overall progress bar
        for idx, interview in enumerate(tqdm(uploaded_videos, desc="🎬 Overall Progress", unit="video", ncols=80), 1):
            if not interview.get('isVideoExist') or not interview.get('recordedVideoUrl'):
                transcriptions.append({
                    'positionId': interview['positionId'],
                    'error': interview.get('error', 'Video not available')
                })
                continue
            
            position_id = interview['positionId']
            video_url = interview['recordedVideoUrl']
            is_external_video = interview.get('isExternal', False)
            
            try:
                print(f'\n┌─ Video {position_id}/{len(uploaded_videos)} ─{"─"*50}┐')
                
                with processing_lock:
                    processing_status[session_id] = {
                        'status': 'processing',
                        'progress': f'{position_id}/{len(uploaded_videos)}',
                        'current_video': position_id,
                        'message': f'Processing video {position_id}/{len(uploaded_videos)}...'
                    }
                
                video_start = time.time()
                
                # Step 0: Get video file (download if external, or get local path)
                if is_external_video:
                    print(f'│ 0️⃣  DOWNLOAD FROM URL')
                    print(f'│    URL: {video_url[:60]}...')
                    with processing_lock:
                        processing_status[session_id]['message'] = f'Downloading video {position_id}...'
                
                local_file = get_video_file(video_url, position_id, is_external_video)
                file_size_mb = os.path.getsize(local_file) / (1024 * 1024)
                
                # Step 1: Transcribe
                print(f'│ 1️⃣  TRANSCRIPTION ({file_size_mb:.1f} MB)')
                with processing_lock:
                    processing_status[session_id]['message'] = f'Transcribing video {position_id}...'
                
                transcription_text = transcribe_video(local_file)
                transcribe_time = time.time() - video_start
                
                # Step 2: Translate
                print(f'│ 2️⃣  TRANSLATION')
                translate_start = time.time()
                with processing_lock:
                    processing_status[session_id]['message'] = f'Translating video {position_id}...'
                
                transcription_id = translate_to_indonesian(transcription_text)
                translate_time = time.time() - translate_start
                
                # Step 3: Save
                print(f'│ 3️⃣  SAVING FILES')
                trans_fname = f"transcription_pos{position_id}_{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}.txt"
                trans_path = os.path.join(TRANSCRIPTION_DIR, trans_fname)
                
                with open(trans_path, 'w', encoding='utf-8') as f:
                    f.write(f"Candidate: {candidate_name}\n")
                    f.write(f"Position ID: {position_id}\n")
                    f.write(f"Video URL: {video_url}\n")
                    f.write(f"Source: {'External URL (downloaded)' if is_external_video else 'Local upload'}\n")
                    f.write(f"Transcribed at: {datetime.now(timezone.utc).isoformat()}\n")
                    f.write(f"Model: faster-whisper large-v3\n")
                    f.write(f"Processing time: {transcribe_time:.1f}s\n")
                    f.write(f"\n{'='*50}\n")
                    f.write(f"ENGLISH TRANSCRIPTION:\n")
                    f.write(f"{'='*50}\n\n")
                    f.write(transcription_text)
                    f.write(f"\n\n{'='*50}\n")
                    f.write(f"INDONESIAN TRANSLATION (DeepL):\n")
                    f.write(f"{'='*50}\n\n")
                    f.write(transcription_id)
                
                transcription_url = f"{base_url}/transcriptions/{trans_fname}"
                
                # Generate assessment
                assessment = generate_dummy_assessment(transcription_text, position_id, transcription_id)
                
                assessment_results.append({
                    "id": position_id,
                    "result": assessment
                })
                
                transcriptions.append({
                    'positionId': position_id,
                    'videoUrl': video_url,
                    'transcription': transcription_text,
                    'transcription_id': transcription_id,
                    'transcriptionUrl': transcription_url,
                    'transcriptionFile': trans_fname,
                    'assessment': assessment
                })
                
                # Delete video file (both downloaded and uploaded)
                if os.path.exists(local_file):
                    os.remove(local_file)
                    action = "Downloaded video" if is_external_video else "Video"
                    print(f'│ 🗑️  {action} deleted ({file_size_mb:.1f} MB freed)')
                
                total_time = time.time() - video_start
                print(f'│ ⏱️  Total: {total_time:.1f}s (Transcribe: {transcribe_time:.1f}s | Translate: {translate_time:.1f}s)')
                print(f'│ 📊 Assessment: {assessment["keputusan_akhir"]} ({assessment["penilaian_akhir"]}/5)')
                print(f'└─{"─"*68}┘')
                
                # Cleanup
                import gc
                gc.collect()
                
            except Exception as e:
                print(f'│ ❌ ERROR: {str(e)}')
                print(f'└─{"─"*68}┘')
                
                transcriptions.append({
                    'positionId': position_id,
                    'videoUrl': video_url,
                    'error': str(e)
                })
                
                # Clean up failed download/upload
                try:
                    if 'local_file' in locals() and os.path.exists(local_file):
                        os.remove(local_file)
                except:
                    pass
        
        # Save final results
        if assessment_results:
            results_json = {
                "success": True,
                "name": candidate_name,
                "session": session_id,
                "content": assessment_results,
                "metadata": {
                    "total_videos": len(uploaded_videos),
                    "successful_videos": len(assessment_results),
                    "processed_at": datetime.now(timezone.utc).isoformat(),
                    "model": "faster-whisper large-v3",
                    "videos_deleted": True,
                    "source": "external_urls" if is_external else "local_uploads",
                    "translation_provider": "DeepL",
                    "translation_language": "Indonesian (ID)"
                }
            }
            
            results_filename = f"{session_id}.json"
            results_path = os.path.join(RESULTS_DIR, results_filename)
            
            with open(results_path, 'w', encoding='utf-8') as f:
                json.dump(results_json, f, ensure_ascii=False, indent=2)
            
            results_url = f"{base_url}/results/{results_filename}"
            print(f'\n💾 Results saved: {results_url}')
        
        successful_count = sum(1 for t in transcriptions if 'transcription' in t)
        
        with processing_lock:
            processing_status[session_id] = {
                'status': 'completed',
                'result': {
                    'success': True,
                    'transcriptions': transcriptions,
                    'processed_videos': len(transcriptions),
                    'successful_videos': successful_count,
                    'failed_videos': len(transcriptions) - successful_count,
                    'results_url': f"{base_url}/results/{session_id}.json" if assessment_results else None
                }
            }
        
        print(f'\n{"="*70}')
        print(f'✅ SESSION COMPLETED ({source_type})')
        print(f'   Success: {successful_count}/{len(transcriptions)} videos')
        print(f'{"="*70}\n')
        
    except Exception as e:
        import traceback
        print(f'\n❌ SESSION ERROR:\n{traceback.format_exc()}')
        
        with processing_lock:
            processing_status[session_id] = {
                'status': 'error',
                'error': str(e),
                'error_detail': traceback.format_exc()
            }

# ENDPOINTS

@app.post('/upload')
async def receive_videos_and_process(
    request: Request,
    candidate_name: str = Form(...),
    videos: List[UploadFile] = File(...)
):
    """Upload videos and start background transcription"""
    session_id = uuid.uuid4().hex
    print(f'\n🔵 NEW UPLOAD REQUEST - Session: {session_id}')
    print(f'   Candidate: {candidate_name}')
    print(f'   Videos: {len(videos)} file(s)')
    
    # Initialize status FIRST
    with processing_lock:
        processing_status[session_id] = {
            'status': 'uploading',
            'progress': '0/0',
            'message': 'Uploading videos...'
        }
    
    try:
        # 1. Upload semua video (fast)
        base_url = str(request.base_url).rstrip('/')
        uploaded_videos = []
        
        print(f'\n📤 Uploading {len(videos)} video(s)...')
        for idx, video in enumerate(videos, 1):
            try:
                ext = os.path.splitext(video.filename)[1] or '.webm'
                safe_name = f"{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}{ext}"
                dest_path = os.path.join(UPLOAD_DIR, safe_name)
                
                # Update upload progress
                with processing_lock:
                    processing_status[session_id]['message'] = f'Uploading video {idx}/{len(videos)}...'
                    processing_status[session_id]['progress'] = f'{idx}/{len(videos)}'
                
                with open(dest_path, 'wb') as buffer:
                    shutil.copyfileobj(video.file, buffer)
                
                file_url = f"{base_url}/uploads/{safe_name}"
                uploaded_videos.append({
                    'positionId': idx,
                    'isVideoExist': True,
                    'recordedVideoUrl': file_url,
                    'filename': safe_name
                })
                print(f'   ✅ Uploaded: {safe_name}')
                
            except Exception as e:
                print(f'   ❌ Failed: {str(e)}')
                uploaded_videos.append({
                    'positionId': idx,
                    'isVideoExist': False,
                    'recordedVideoUrl': None,
                    'error': str(e)
                })
        
        # 2. Update status to processing
        with processing_lock:
            processing_status[session_id] = {
                'status': 'processing',
                'progress': '0/' + str(len(uploaded_videos)),
                'message': 'Starting transcription...',
                'uploaded_videos': len(uploaded_videos)
            }
        
        # 3. Start background thread
        thread = th.Thread(
            target=process_videos_unified,  # CHANGED: use unified function
            args=(session_id, candidate_name, uploaded_videos, base_url),
            daemon=True
        )
        thread.start()
        
        print(f'✅ Upload complete. Background thread started.')
        print(f'📤 Returning immediate response with session_id: {session_id}')
        
        # 4. RETURN IMMEDIATELY - no waiting!
        return JSONResponse(
            content={
                'success': True,
                'session_id': session_id,
                'message': 'Videos uploaded successfully. Processing started.',
                'uploaded_videos': len(uploaded_videos)
            },
            status_code=200,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST, GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )
        
    except Exception as e:
        import traceback
        error_detail = traceback.format_exc()
        print(f'❌ Error:\n{error_detail}')
        
        # Update status to error
        with processing_lock:
            processing_status[session_id] = {
                'status': 'error',
                'error': str(e),
                'error_detail': error_detail
            }
        
        return JSONResponse(
            content={
                'success': False,
                'session_id': session_id,
                'error': str(e)
            },
            status_code=500,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST, GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )

@app.post('/upload_json')
async def receive_json_and_process(
    request: Request,
    background_tasks: BackgroundTasks
):
    """Receive JSON payload with external video URLs and process"""
    session_id = uuid.uuid4().hex
    
    try:
        # Parse JSON body
        json_data = await request.json()
        
        print(f'\n🔵 NEW JSON UPLOAD - Session: {session_id}')
        
        # Validate structure
        if not json_data.get('success'):
            raise HTTPException(status_code=400, detail="Invalid JSON: success flag missing")
        
        data = json_data.get('data', {})
        candidate = data.get('candidate', {})
        candidate_name = candidate.get('name')
        
        if not candidate_name:
            raise HTTPException(status_code=400, detail="Invalid JSON: candidate name missing")
        
        review_checklists = data.get('reviewChecklists', {})
        interviews = review_checklists.get('interviews', [])
        
        if not interviews or not isinstance(interviews, list):
            raise HTTPException(status_code=400, detail="Invalid JSON: interviews array missing")
        
        print(f'   Candidate: {candidate_name}')
        print(f'   Videos: {len(interviews)} file(s)')
        
        # Initialize status
        with processing_lock:
            processing_status[session_id] = {
                'status': 'processing',
                'progress': '0/0',
                'message': 'Starting video download and processing...'
            }
        
        # Prepare video list for processing
        base_url = str(request.base_url).rstrip('/')
        uploaded_videos = []
        
        for interview in interviews:
            position_id = interview.get('positionId')
            video_url = interview.get('recordedVideoUrl')
            is_exist = interview.get('isVideoExist', False)
            
            if not is_exist or not video_url:
                uploaded_videos.append({
                    'positionId': position_id,
                    'isVideoExist': False,
                    'recordedVideoUrl': None,
                    'error': 'Video not available or URL missing'
                })
            else:
                uploaded_videos.append({
                    'positionId': position_id,
                    'isVideoExist': True,
                    'recordedVideoUrl': video_url,
                    'isExternal': True  # Mark as external URL
                })
        
        # Start background processing using UNIFIED function
        thread = th.Thread(
            target=process_videos_unified,  # CHANGED: use unified function
            args=(session_id, candidate_name, uploaded_videos, base_url),
            daemon=True
        )
        thread.start()
        
        print(f'✅ JSON received. Background download & processing started.')
        print(f'📤 Returning immediate response with session_id: {session_id}')
        
        return JSONResponse(
            content={
                'success': True,
                'session_id': session_id,
                'message': 'JSON received. Video download and processing started.',
                'videos_to_process': len(uploaded_videos)
            },
            status_code=200,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )
        
    except HTTPException as he:
        raise he
    except Exception as e:
        import traceback
        error_detail = traceback.format_exc()
        print(f'❌ JSON processing error:\n{error_detail}')
        
        with processing_lock:
            processing_status[session_id] = {
                'status': 'error',
                'error': str(e),
                'error_detail': error_detail
            }
        
        return JSONResponse(
            content={
                'success': False,
                'session_id': session_id,
                'error': str(e)
            },
            status_code=500,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'POST, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )

@app.get('/status/{session_id}')
async def get_processing_status(session_id: str):
    """Check processing status"""
    with processing_lock:
        if session_id not in processing_status:
            return JSONResponse(
                {
                    'status': 'not_found',
                    'message': 'Session not found'
                }, 
                status_code=404,
                headers={
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'GET, OPTIONS',
                    'Access-Control-Allow-Headers': '*',
                    'Cache-Control': 'no-cache, no-store, must-revalidate',
                }
            )
        
        status_copy = processing_status[session_id].copy()
    
    # Add redirect URL if completed
    if status_copy.get('status') == 'completed':
        status_copy['redirect'] = f"halaman_dasboard.html?session={session_id}"
    
    return JSONResponse(
        status_copy,
        headers={
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'GET, OPTIONS',
            'Access-Control-Allow-Headers': '*',
            'Cache-Control': 'no-cache, no-store, must-revalidate',
        }
    )

@app.get('/results/{session_id}')
async def get_results(session_id: str):
    """Get assessment results for a session"""
    results_filename = f"{session_id}.json"
    results_path = os.path.join(RESULTS_DIR, results_filename)
    
    if not os.path.exists(results_path):
        return JSONResponse(
            {
                'success': False,
                'message': 'Results not found for this session',
                'session_id': session_id
            },
            status_code=404,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )
    
    try:
        with open(results_path, 'r', encoding='utf-8') as f:
            results_data = json.load(f)
        
        return JSONResponse(
            results_data,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
                'Cache-Control': 'no-cache, no-store, must-revalidate',
            }
        )
    except Exception as e:
        return JSONResponse(
            {
                'success': False,
                'message': f'Error reading results: {str(e)}',
                'session_id': session_id
            },
            status_code=500,
            headers={
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'GET, OPTIONS',
                'Access-Control-Allow-Headers': '*',
            }
        )

@app.get('/')
async def index():
    return {
        'message': 'AI Interview Assessment System',
        'model': 'faster-whisper large-v3',
        'accuracy': '98%+ for clear English speech',
        'speed': '4-5x faster than standard Whisper',
        'endpoints': {
            'upload': 'POST /upload',
            'status': 'GET /status/{session_id}',
            'results': 'GET /results/{session_id}',
            'test_form': 'GET /upload_form'
        }
    }

@app.get('/upload_form')
async def upload_form():
    html = '''
    <html>
      <head><meta charset="utf-8"><title>Upload Videos - AI Interview System</title></head>
      <body style="font-family: Arial, sans-serif; padding: 20px;">
        <h2>🎙️ AI Interview Assessment System</h2>
        <p><strong>Model:</strong> faster-whisper large-v3 (Maximum Accuracy)</p>
        <p><strong>Accuracy:</strong> ~98% for clear English speech</p>
        <p><strong>Speed:</strong> 4-5x faster than standard Whisper</p>
        <hr>
        <h3>Upload Interview Videos</h3>
        <form action="/upload" enctype="multipart/form-data" method="post">
          <label>Candidate Name: <input name="candidate_name" type="text" required style="padding: 5px; width: 300px;" /></label><br><br>
          <label>Select Videos: <input name="videos" type="file" accept="video/*" multiple required /></label><br><br>
          <button type="submit" style="padding: 10px 20px; background: #007bff; color: white; border: none; cursor: pointer;">Upload & Transcribe</button>
        </form>
        <p style="color: #666; font-size: 14px;">Tip: Clear audio will result in better transcription accuracy.</p>
      </body>
    </html>
    '''
    return HTMLResponse(content=html, status_code=200)

print('✅ FastAPI app defined successfully')
print('ℹ️  Run next cell to start the server')

C:\Users\Daffa\AppData\Roaming\Python\Python313\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
C:\Users\Daffa\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Daffa\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



📥 Loading Whisper model...
ℹ️  Using faster-whisper "large-v3" model
   This is the MOST ACCURATE model available
   Speed: 4-5x faster than openai-whisper
   Accuracy: ~98% for clear English speech
   First run will download ~3GB model...

🎯 Configuration:
   Device: CPU
   Compute Type: int8
✅ Whisper model loaded successfully

✅ DeepL translator initialized successfully

✅ Whisper model loaded successfully

✅ DeepL translator initialized successfully



In [ ]:
# Jalankan server uvicorn di dalam notebook (tanpa ngrok)
import nest_asyncio
import uvicorn
import threading

nest_asyncio.apply()
PORT = 8888

# Hentikan server sebelumnya jika ada
if 'server_thread' in globals() and server_thread is not None:
    try:
        print('⏸️  Stopping previous server...')
        if 'server' in globals() and server is not None:
            server.should_exit = True
        # Tunggu thread selesai (dengan timeout)
        if server_thread.is_alive():
            server_thread.join(timeout=2)
        print('✅ Previous server stopped.')
    except Exception as e:
        print(f'⚠️  Error stopping previous server: {e}')

# Buat server instance baru dengan log level yang lebih rendah
config = uvicorn.Config(
    app=app, 
    host='0.0.0.0', 
    port=PORT, 
    log_level='warning',  # Kurangi verbosity untuk menghindari duplikasi log
    access_log=False  # Nonaktifkan access log di console
)
server = uvicorn.Server(config=config)

# Fungsi untuk menjalankan server di thread
def run_server_in_thread():
    import asyncio
    # Buat event loop baru untuk thread ini
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        loop.run_until_complete(server.serve())
    except Exception as e:
        print(f'❌ Server error: {e}')
    finally:
        loop.close()

# Jalankan server di background thread
server_thread = threading.Thread(target=run_server_in_thread, daemon=True)
server_thread.start()

print('━' * 60)
print('🚀 Server started successfully!')
print(f'📍 Local URL: http://127.0.0.1:{PORT}')
print(f'📍 Network URL: http://0.0.0.0:{PORT}')
print(f'🔧 Endpoints:')
print(f'   - POST /upload       (upload videos & process)')
print(f'   - POST /upload_json  (upload JSON & download videos)')
print(f'   - GET  /status/{{id}}  (check processing status)')
print(f'   - GET  /results/{{id}} (get assessment results)')
print(f'   - GET  /upload_form  (test form)')
print('ℹ️  Use Interrupt Kernel to stop the server')
print('━' * 60)

## System Information

### Whisper Model
- **Library**: `faster-whisper` (optimized implementation)
- **Model**: `large-v3` (most accurate available)
- **Accuracy**: ~98% for clear English speech
- **Speed**: 4-5x faster than `openai-whisper`

### Translation
- **Provider**: DeepL API
- **Target Language**: Indonesian (ID)
- **Source Language**: English (EN)
- **Character Limit**: 5,000 per chunk
- **Setup**: Set `DEEPL_API_KEY` in cell 3
- **Get API Key**: https://www.deepl.com/pro-api (Free tier: 500,000 chars/month)

### Performance
- **Device**: Automatically detects CUDA GPU (if available) or CPU
- **Compute Type**: 
  - GPU: `float16` (faster with high accuracy)
  - CPU: `int8` (optimized for CPU)
- **VAD Filter**: Enabled (skips silence for efficiency)

### Settings
- **Beam Size**: 5 (higher = more accurate)
- **Best Of**: 5 (samples multiple candidates)
- **Patience**: 2.0 (thorough beam search)
- **Temperature**: 0.0 (deterministic output)
- **Context**: Uses previous text for better accuracy

### Storage Management
- **Auto-delete videos**: ✅ Videos are automatically deleted after successful transcription
- **Storage saved**: Only transcriptions and results are kept
- **Safety**: Deletion only happens after successful transcription
- **Error handling**: If deletion fails, processing continues normally

### Endpoints
- `POST /upload` - Upload videos and start transcription
- `GET /status/{session_id}` - Check processing status
- **`GET /results/{session_id}`** - **Get assessment results**
- `GET /upload_form` - Test form interface
- `GET /` - System information

### Files
- ~~Uploaded videos: `uploads/`~~ (deleted after transcription) ♻️
- Transcriptions: `transcriptions/` ✅ (includes English + Indonesian)
- **Assessment results: `results/`** ✅

### Assessment Data Structure
```json
{
  "success": true,
  "name": "Candidate Name",
  "session": "session_id_here",
  "content": [
    {
      "id": 1,
      "result": {
        "penilaian": { ... },
        "penilaian_akhir": 5,
        "cheating_detection": "Tidak",
        "alasan_cheating": "Tidak ada indikasi kecurangan",
        "analisis_non_verbal": "Lancar dan tidak mencurigakan",
        "keputusan_akhir": "Lulus",
        "transkripsi": "Full English transcription...",
        "transkripsi_id": "Transkripsi lengkap dalam Bahasa Indonesia...",
        "metadata": {
          "word_count": 150,
          "char_count": 800,
          "processed_at": "2024-01-01T12:00:00Z",
          "translation_available": true
        }
      }
    }
  ],
  "metadata": {
    "total_videos": 5,
    "successful_videos": 5,
    "processed_at": "2024-01-01T12:00:00Z",
    "model": "faster-whisper large-v3",
    "videos_deleted": true,
    "translation_provider": "DeepL",
    "translation_language": "Indonesian (ID)"
  }
}
```

### Notes
- Assessment scores are currently **dummy data** for testing
- Replace `generate_dummy_assessment()` with real AI analysis later
- Results are saved automatically after transcription completes
- **Original video files are deleted after transcription to save storage**
- **English transcription + Indonesian translation included**
- DeepL API key required for translation (free tier available)
- Access via: `http://127.0.0.1:8888/results/{session_id}`

### DeepL Setup
1. Sign up at https://www.deepl.com/pro-api
2. Get your free API key (500,000 chars/month)
3. Set `DEEPL_API_KEY` in cell 3
4. Restart kernel and run all cells